In [26]:
%cd /home/jovyan/onceuponai/websites/src/stories-fantasy-shop-assistant/notebooks
from IPython.core.display import HTML
HTML("""
<link rel="stylesheet" type="text/css" href="//fonts.googleapis.com/css?family=Bangers" />
<style>
h1 {font-family: Bangers !important; font-size: 150px !important}
h2 {font-family: Bangers !important; font-size: 120px !important}
h3 {font-family: Bangers !important; font-size: 100px !important}
h4 {font-family: Bangers !important; font-size: 80px !important}
h5 {font-family: Bangers !important; font-size: 50px !important}
h6 {font-family: Bangers !important; font-size: 30px !important}
.txt7 {font-family: Bangers !important; font-size: 70px !important}
.txt6 {font-family: Bangers !important; font-size: 60px !important}
.txt5 {font-family: Bangers !important; font-size: 50px !important}
.txt4 {font-family: Bangers !important; font-size: 40px !important}
.txt3 {font-family: Bangers !important; font-size: 30px !important}
.cap {font-family: Bangers !important; font-size: 30px !important}
.controls
{
    display: none !important;
}

.slide-number
{
    display: none !important;
}
.inline-block {
   display: inline-block;
}

.backgrounds {
   background-color: #fff !important;
}

.controls {
  display: none !important;
}
</style>
""")

import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

/home/jovyan/onceuponai/websites/src/stories-fantasy-shop-assistant/notebooks


In [ ]:
!pip install deltalake polars

# LLMs

In [1]:
import onceuponai

bielik = onceuponai.llms.Quantized("speakleash/Bielik-7B-Instruct-v0.1-GGUF", \
                              "bielik-7b-instruct-v0.1.Q4_K_S.gguf", \
                              "speakleash/Bielik-7B-Instruct-v0.1", "cuda")

In [13]:
import onceuponai

bielik = onceuponai.llms.Quantized("speakleash/Bielik-7B-Instruct-v0.1-GGUF", \
                              "file:///home/jovyan/rust-src/Bielik/Bielik-7B-Instruct-v0.1.Q4_K_S.gguf", \
                              "speakleash/Bielik-7B-Instruct-v0.1", "cuda")

In [ ]:
res = await bielik.invoke("Jak ugotować żurek ?", 1000)
res

In [1]:
import onceuponai

krakowiak = onceuponai.llms.Quantized("szymonrucinski/Krakowiak-7B-v2", \
                              "file:///home/jovyan/rust-src/Bielik/Krakowiak-7B-v2.Q4_K_S.gguf", \
                              "szymonrucinski/Krakowiak-7B-v2", "cuda")

In [2]:
res = await krakowiak.invoke("Jak ugotować żurek ?", 1000)
res

'\n groszek, krwi, cebula, koper, mięta, pietruszka, liście laurowe.\nWiększość potraw można ugotować w domu. Dlatego teraz oferujemy Ci szczegółowy przepis na ugotowanie żurku – to jest łatwy do wykonania, ale bardzo smaczne danie.\nDla gotowania mąki stosuj się do innej recepcji “Smażone ziemniaki” lub wystarczy włożyć 100 gram na 1 kubak żurku. Rozgnieść tłuczko między dłońmi aż uzyska ciasto.\nWedług innych przepisów, w pierwszej kolejności rozpuścić solę ziemniaczaną w gorącej wodzie i zeskrobać skórkę żurka, ale jak już wspomniano powyżej, jest to bardzo krzywdne dla żurkowca. Uwolnij ciepło żurku przez 5 minut w lodówce – a następnie umyć go pod temperaturą wrzącej wody (ujmij) – aż wykiwka i nie zapowiada się, a następnie rozgnieść na ciasto. Jeśli masz w domu uniwersalne tłuczko, potrwa to 2 minuty, ale mogą być bardziej skomplikowane i czasochłonne niż utrzymywanie żurka na piekarniku podczas gotowania żurkowego 10 minut lub więcej. Jeśli nie masz uniwersalnego tłuczka, można

In [1]:
import onceuponai

curie = onceuponai.llms.Quantized("szymonrucinski/Curie-7B-v1", \
                              "file:///home/jovyan/rust-src/Bielik/Curie-7B-v1.Q3_K.gguf", \
                              "szymonrucinski/Curie-7B-v1", "cuda")

In [2]:
res = await curie.invoke("Jak ugotować rosół ?", 1000)
res

'Prosty i smaczny przepis krok po kroku Jak przyrządzić rosół? Przepis na pysznego, ciepłego obiadu w ogóle nie skomplikowany. Zgodnie z przepisem rosół przyrasta na rozgrzanym opalonym olejem, najpierw grillowany sezonowa wieprzowa karkówka lub rącznik pogryziony do 10 minut, następnie grzyby i marchewki. Na tym bazie przygotowuje się rosół w które można wcześniej dopisać potrawy tzw. na zimno. Zawikł w polskiej kuchni od wieków, a jedynie niedawno powoli wyparły go szpicbordy z ziemi odmiany z jarzyna i plastrą szynkę. Najlepiej rosół przyrządzić na ogniu. Znajduje się w nim wiele smakowitej skóra, ślizg, tłuszczu (bardzo jest go więcej niż w przypadku koncentratu jarzynowego), który podawany na obiadowe danie. Smaczny i niezrównany, a przy okazji po prostu idealnie smakujący. Rosół będący gotowaniną, czyli jedną z największych tradycyjnych potraw w kuchni polskiej, to obiad dla gości oraz rodziny. Można jeść je świeże lub zamrażone. Podobnie jak wiele innych potraw na bazie mięsa, r

In [3]:
import onceuponai

await onceuponai.sleep(2)

## Prepare data

In [ ]:
import yaml
from deltalake.writer import write_deltalake
import pandas as pd
import polars as pl

data = yaml.safe_load(open("recipes.yaml", "r"))

prep={}


df = pl.DataFrame(data["Recipes"])
df.to_pandas()

# Embeddings

In [1]:
import onceuponai

a = onceuponai.embeddings.E5("intfloat/multilingual-e5-small")

In [2]:
a.embed(["witaj przygodo"])

[[0.03098888322710991,
  -0.01199488528072834,
  -0.07422228902578354,
  -0.06891388446092606,
  0.05637747794389725,
  -0.03292524069547653,
  0.05126916617155075,
  0.03614596650004387,
  0.023003606125712395,
  0.006733578629791737,
  0.034447889775037766,
  0.02289721742272377,
  0.05806450918316841,
  -0.04183975234627724,
  -0.02903415448963642,
  0.03861762955784798,
  0.06157746538519859,
  -0.06715274602174759,
  -0.038856398314237595,
  -0.029826771467924118,
  0.039446957409381866,
  -0.03411424532532692,
  -0.06606124341487885,
  0.04624686390161514,
  0.05065515637397766,
  0.03813409432768822,
  -0.0525520034134388,
  0.06392091512680054,
  0.037307772785425186,
  -0.05885927751660347,
  -0.05158555135130882,
  -0.05430685728788376,
  0.03010113537311554,
  -0.03858902305364609,
  0.05888715386390686,
  0.08750040084123611,
  -0.02383234351873398,
  -0.03985855355858803,
  0.0594383180141449,
  -0.05335207283496857,
  -0.009211460128426552,
  0.019977519288659096,
  0.020

In [ ]:
items = []
for combined_text in df['Recipe']:
    emb = a.embed([combined_text])
    items.append({"item": combined_text, "vector": emb[0]})

items

# Fake words

In [ ]:
import random

subjects = [
    "The mischievous squirrel", "The colorful birds", "The playful puppies", 
    "The dedicated teacher", "The curious boy", "The brilliant inventor", "Girl", "Cat", "Mouse", "Dog"
]

verbs = [
    "danced", "explored", "built", "discovered", "imagined", 
    "sang", "cooked", "programmed","run", "sing", "walk", 
]

objects = ["the sandwich", "a brilliant app", "over the fence", "a beautiful song", "in the sky"]

for i in range(300):
    phrase = random.choice(subjects) + " " + random.choice(verbs) + " " + random.choice(objects)
    emb = a.embeddings([phrase])
    items.append({"item": phrase, "vector": emb[0]})

# Lance DB

In [ ]:
import os
import getpass

print("ACCOUNT NAME")
os.environ["AZURE_STORAGE_ACCOUNT_NAME"]=getpass.getpass()

print("ACCOUNT KEY")
os.environ["AZURE_STORAGE_ACCOUNT_KEY"]=getpass.getpass()

In [ ]:
import lancedb

db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.create_table("recipes_vectors", data=items)

In [ ]:
res = await tbl.create_index("vector")

# Search

In [6]:
import lancedb

emb = a.embed(["Chciałbym zrobić żurek"])
db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.open_table("recipes_vectors")
res = await tbl.query().nearest_to(emb[0]).limit(1).to_pandas()
res

,item,vector,_distance
0,Jak gotować żurek staropolski z jajkiem i biał...,"[0.03319656, -0.016286198, -0.041886758, -0.05...",0.282364


In [22]:
import lancedb

db = await lancedb.connect_async("az://recipesv2/vectors2")
tbl = await db.open_table("recipes_vectors")
template = """[INST]Skorzystaj z poniższych fragmentów kontekstu, aby odpowiedzieć na pytanie na końcu. 
Jeśli nie znasz odpowiedzi, po prostu powiedz, że nie wiesz, nie próbuj wymyślać odpowiedzi. 

Kontekst: 
{context}

Pytanie:
{question}[/INST]"""

async def chat(lancedb_table, prompt_template, embedding_model, llm_model, prompt):
    emb = embedding_model.embed([prompt])
    res = await lancedb_table.query().nearest_to(emb[0]).limit(1).to_pandas()
    context = res.to_dict()["item"][0]
    prompt = prompt_template.replace("{context}", context).replace("{question}", prompt)
    return await llm_model.invoke(prompt, 1000)
    

In [26]:
await chat(tbl, template, a, bielik, "Jak ugotować żurek ?")

' Aby ugotować żurek staropolski, zacznij od przygotowania zakwasu na trzy dni przed świętem. Następnie gotuj kości wieprzowe lub wędzonkę dla gęstości smaku w garze razem z korzeniami i wodą. Przygotuj oddzielnie białą kiełbasę, aż będzie miękka, przecedź wywar i wrzuć do niego zakwas. Przypraw zupę solą, majerankiem, i czosnkiem. Przed podaniem dodać pokrojone kiełbaski, jajka ugotowane na twardo i posypać świeżym majerankiem. Serwuj go w glinianych misach z chlebem razowym obok do zanurzania.'

In [24]:
await chat(tbl, template, a, bielik, "W jakiej temperaturze należy piec rybę ?")

' 200°C'

In [25]:
await chat(tbl, template, a, bielik, "Ile czasu trzeba piec kaczkę ?")

' 1,5 do 2 godzin'